# Jetpack Compose - LiveData - Podstawy

W tej aplikacji przyjrzymy się zastosowaniu `LiveData` - części architektury **MVVM**.

#### **UWAGA**
W aplikacjach opartych na **JetpackCompose** wykorzystuje się `State` (który widzieliśmy w poprzednim przykładzie), `StateFlow`, lub `ComposeFlow` (które poznamy na kolejnych zajęciach). Również można tworzyć aplikacje oparte na `LiveData` lecz jest to rzadziej spotykane. W praktyce posiadając `LiveData` w aplikacji konwertujemy go do `State` przez metodę `observeAsState`, z którą zapoznamy się w tym przykładzie.

<img src="https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExMDExYnllNDZwZW40NjAxbzl4ZTNlZXN0dHZ1aXE5MjFha2g2ZXV1ayZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/xYi3Uxdr0LCyxmoDJO/giphy.gif" width="200" />

`LiveData` jest częścią bibliotek Androida i jest obiektem, który przechowuje dane **obserwowane** przez komponenty aplikacji, takie jak aktywności, fragmenty czy `ViewModel`. Jest zaprojektowany tak, aby dostarczać reaktywne i cykliczne powiadomienia o zmianach danych.

**Reaktywność** to podejście w programowaniu, które koncentruje się na tym, aby system reagował na zmiany i propagował te zmiany w sposób automatyczny. W kontekście aplikacji, reaktywność odnosi się do zdolności systemu do dynamicznego reagowania na zmiany danych i propagowania tych zmian do odpowiednich komponentów. 

Opiera się na obsłudze zdarzeń, które są generowane w systemie w wyniku **zmian w danych**. Zdarzenia te mogą być przekształcane i łączone za pomocą różnych operacji, tworząc **strumienie danych**. Strumienie są sekwencją wartości, które mogą być emitowane i subskrybowane (obserwowane) przez komponenty. Automatycznie propaguje zmiany w danych do komponentów, które subskrybują te dane. Oznacza to, że komponenty **nie muszą** ręcznie monitorować i aktualizować danych, ponieważ system sam zarządza tym procesem.

- Obserwowanie zmian - Komponenty mogą subskrybować (obserwować) `LiveData`, aby otrzymywać powiadomienia o zmianach danych. Kiedy wartość zostanie zaktualizowana, subskrybenci (obserwatorzy) otrzymają aktualizację i będą mogli zareagować na nową wartość.
- Bezpieczeństwo wątków - `LiveData` automatycznie zarządza wątkami, zapewnia, otrzymanie powiadomień na wątku `UI` (**głównym wątku**). Jeśli wartość `LiveData` zostanie zmieniona w wątku innym niż wątek główny, powiadomienia zostaną dostarczone do obserwatorów w wątku głównym, co ułatwia aktualizację interfejsu użytkownika.
- Unikanie wycieków pamięci: `LiveData` jest **świadome cyklu życia komponentów** i automatycznie **zatrzymuje obserwację**, gdy komponenty, które go subskrybują, są w **stanie nieaktywnym**, takim jak aktywność w stanie pauzy lub fragment w stanie `detach`. Dzięki temu unika się wycieków pamięci, które mogą wystąpić, gdy komponenty trzymają referencje do innych komponentów poza ich cyklem życia.

`LiveData` jest użyteczne w przypadku komunikacji między **warstwami aplikacji**, takimi jak `ViewModel` i interfejs użytkownika, ponieważ umożliwia bezpośrednie powiązanie danych z widokami (elementami ui), eliminując potrzebę ręcznej synchronizacji danych.

`LiveData` jest bardziej odpowiednie do **tradycyjnych** (gdzie ui tworzony jest w oparciu o `View` z `Fragment`) aplikacji Android. `State` natomiast jest dedykowane dla aplikacji opartych na **Jetpack Compose**, które używają **deklaratywnego podejścia** do budowania `UI`. Oba mechanizmy dostarczają reaktywności i automatycznego odświeżania widoku w odpowiedzi na zmiany danych.

Utworzymy prosty licznik, w którym wartość licznika będzie obserwowana. komponent odpowiedzialny za renderowanie ekranu będzie reagował automatycznie na zmiany tej wartości.

Aby skorzystać z `LiveData` i `ViewModel` musimy dodać odpowiednie zależności.

do bloku `dependencies`

```kotlin
    implementation (libs.androidx.lifecycle.viewmodel.compose)
    implementation (libs.androidx.lifecycle.livedata.ktx)
    implementation (libs.androidx.runtime)
    implementation (libs.androidx.runtime.livedata)
    ...
```

do pliku `libs.versions.toml`
```kotlin
[versions]
coreKtx = "1.13.1"
lifecycleRuntimeKtx = "2.8.4"
runtime = "1.6.8"
...

[libraries]
androidx-core-ktx = { group = "androidx.core", name = "core-ktx", version.ref = "coreKtx" }
androidx-lifecycle-livedata-ktx = { module = "androidx.lifecycle:lifecycle-livedata-ktx", version.ref = "lifecycleRuntimeKtx" }
androidx-lifecycle-viewmodel-compose = { module = "androidx.lifecycle:lifecycle-viewmodel-compose", version.ref = "lifecycleRuntimeKtx" }
androidx-runtime = { module = "androidx.compose.runtime:runtime", version.ref = "runtime" }
androidx-runtime-livedata = { module = "androidx.compose.runtime:runtime-livedata", version.ref = "runtime" }
...
```

Rozpocznijmy od utworzenia `ViewModel`

### ViewModel

In [ ]:
class CounterViewModel : ViewModel() {
    private var _counter = MutableLiveData(0)
    val counter: LiveData<Int>
        get() = _counter

    fun increase(){
        _counter.value = _counter.value?.inc()
    }

    fun decrease(){
        _counter.value = _counter.value?.dec()
    }

    fun clear(){ _counter.value = 0 }
}

- `private var _counter = MutableLiveData(0)` - `MutableLiveData` jest klasą służącą do przechowywania i zarządzania danymi reaktywnymi. pole przechowuje aktualną wartość licznika i jest zainicjalizowane wartością 0. Jest to prywatna wartość wspierająca - mamy do niej dostęp tylko z poziomu `CounterViewModel`. Jeżeli chcemy zmienić wartość z poziomu `CounterScreen`, wywołujemy odpowiednią metodę dostępową, która odpowiednio modyfikuje to pole.
- `val counter: LiveData<Int> get() = _counter` Pole dostarcza dostęp do pola `_counter` tylko do odczytu, umożliwiając obserwowanie jego wartości bez możliwości zmiany. Jest to pole publiczne, które jest częścią interfejsu `CounterViewModel`
- `fun increase()` Metoda dostępowa zwiększająca stan licznika.
- `fun decrease()` Metoda dostępowa zmniejszająca stan licznika.
- `fun clear()` Metoda dostępowa rresetująca stan licznika.

Klasa `CounterViewModel` dostarcza interfejs do zarządzania stanem licznika poprzez metody `increase()`, `decrease()` i `clear()`. Pola `_counter` i `counter` umożliwiają obserwowanie wartości licznika i reagowanie na jego zmiany w innych komponentach są subskrybentami (blokują możliwość bezpośredniej zmiany licznika - do tego służą metody dostępowe).

### CounterScreen

In [ ]:
@Composable
fun CounterScreen() {

    val viewModel: CounterViewModel = viewModel() // zapewnia dostęp do metod dostępowych i danych (tylko do odczytu)
    val counterState = viewModel.counter.observeAsState() // przy każdej zmianie wartości counter,
                                                          // counterState otrzymuje aktualną wartość

    Column(
        verticalArrangement = Arrangement.Center,
        horizontalAlignment = Alignment.CenterHorizontally,
        modifier = Modifier.fillMaxSize()
    ) {
        Spacer(modifier = Modifier.weight(0.3f))
        Text(
            text = counterState.value.toString(),
            fontSize = 250.sp,
            textAlign = TextAlign.Center,
            modifier = Modifier.weight(1f),
        )
        Row(
            verticalAlignment = Alignment.CenterVertically,
            horizontalArrangement = Arrangement.Center,
            modifier = Modifier.fillMaxWidth()
        ) {
            Button(
                modifier = Modifier
                    .weight(.5f)
                    .padding(4.dp),
                shape = RoundedCornerShape(4.dp),
                onClick = {viewModel.increase()} // przycisk zwiększający stan licznika o 1
            ) {
                Text(text = "Increase")
            }
            Button(
                modifier = Modifier
                    .weight(.5f)
                    .padding(4.dp),
                shape = RoundedCornerShape(4.dp),
                onClick = {viewModel.decrease()} // przycisk zmniejszający stan licznika o 1
            ) {
                Text(text = "Decrease")
            }
        }

        Button(
            modifier = Modifier.fillMaxWidth(),
            shape = RectangleShape,
            onClick = {viewModel.clear()} // przycisk ustawiający stan licznika na 0
        ) {
            Text(text = "Reset")
        }
    }
}

Tworzymy dwie zmienne

In [ ]:
val viewModel: CounterViewModel = viewModel()
val counterState = viewModel.counter.observeAsState()

`viewModel` - która przechowuje instancję `CounterViewModel`.
`counterState` który jest obiektem `State` uzyskanym za pomocą funkcji `observeAsState()`. Funkcja `observeAsState()` automatycznie subskrybuje `LiveData` i zwraca jego aktualną wartość w postaci `State`. Dzięki temu, gdy wartość `counter` w `viewModel` ulegnie zmianie, `counterState` zostanie automatycznie zaktualizowane. To spowoduje odświeżenie komponentu, który korzysta z tej wartości.

Za każdym razem, gdy zmieniamy wartość `_counter` za pomocą metod dostępowych (`increase()`, `decrease()`, `clear()`), tekst w polu `Text`
```kotlin
text = counterState.value.toString(),
```
zostanie automatycznie odświeżona.

Możemy przetestować aplikację

<table><tr><td><img src="https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExMDExYnllNDZwZW40NjAxbzl4ZTNlZXN0dHZ1aXE5MjFha2g2ZXV1ayZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/xYi3Uxdr0LCyxmoDJO/giphy.gif" width="150" /></td><td><img src="https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExZWl6emZ3cWxvYzByMDJ1aWdtbG94eTlyZzI0MTh4ejluejg1dnhzNiZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/S3uYswjxuVK1ILsds0/giphy.gif" width="200" /></td></tr></table>